In [1]:
import pandas as pd
from imdb import Cinemagoer, IMDbError
# from difflib import SequenceMatcher
# from googletrans import Translator
from pprint import pprint
from thefuzz import fuzz
import re
from themoviedb import TMDb, aioTMDb
import asyncio
import numpy as np

## READ DATA

In [2]:
customers = pd.read_excel('./DATA SET - VÒNG 1 CUỘC THI DATA GOT TALENT 2023.xlsx', sheet_name='customer')
ticket = pd.read_excel('./DATA SET - VÒNG 1 CUỘC THI DATA GOT TALENT 2023.xlsx', sheet_name='ticket')
film = pd.read_excel('./DATA SET - VÒNG 1 CUỘC THI DATA GOT TALENT 2023.xlsx', sheet_name='film')

In [3]:
customers.shape

(4479, 7)

In [4]:
ticket.shape

(35474, 15)

In [5]:
film.shape

(60, 10)

In [4]:
customers.head()

,customerid,DOB,gender,address,Website,job,industry
0,0000000014,36268,Nữ,hoa khe quan thanh khe,KH0104|0345,student,computer
1,0000000034,35901,Nam,man thai,KH0104|0333,student,health service
2,0000000051,34319,Nam,que son quang nam,KH0104|0255,blue collar,economics
3,0000000081,36472,Nam,truong chinh,KH0104|0293,student,economics
4,0000000098,34608,Nam,cẩm lệ,KH0104|40580,blue collar,health service


## Checking and change type data

In [3]:
customers['DOB']=pd.to_datetime(customers['DOB'], unit='s')

In [4]:
customers

,customerid,DOB,gender,address,Website,job,industry
0,0000000014,1970-01-01 10:04:28,Nữ,hoa khe quan thanh khe,KH0104|0345,student,computer
1,0000000034,1970-01-01 09:58:21,Nam,man thai,KH0104|0333,student,health service
2,0000000051,1970-01-01 09:31:59,Nam,que son quang nam,KH0104|0255,blue collar,economics
3,0000000081,1970-01-01 10:07:52,Nam,truong chinh,KH0104|0293,student,economics
4,0000000098,1970-01-01 09:36:48,Nam,cẩm lệ,KH0104|40580,blue collar,health service
...,...,...,...,...,...,...,...
4474,KH9855766,1970-01-01 09:41:25,Nữ,"A, THUAN PHUOC,HAI CHAU, DA NANG",NaN,specialist,finance
4475,KH9958204,1970-01-01 10:14:52,Nữ,"Phước Mỹ, ĐN",NaN,teenager,NaN
4476,WEBS00000043900,1970-01-01 08:58:31,Nữ,nguyen gian thanh,KHWEBS|00006890,specialist,construction
4477,WEBS00000044909,1970-01-01 09:48:36,Nữ,ngũ hành sơn,KHWEBS|00007899,blue collar,finance


## DEALING WITH NULL DATA AND DUPLICATES DATA

### Customers table

In [5]:
customers.isnull().sum()

customerid       0
DOB              0
gender           0
address          1
Website        536
job              0
industry      1122
dtype: int64

Với cột dữ liệu Job có giá trị là teenager thì trường industry sẽ null -> lý do là bố mẹ hoặc người thân dẫn theo trẻ nhỏ đi xem phim. Hoặc là trẻ em đang ở tuổi vị thành niên tự đi xem.
Có thể fill null bằng "no industry" 


In [6]:
customers.loc[customers['job']=='teenager'].industry.isnull().sum()

1122

In [7]:
customers['industry'].fillna('no industry', inplace=True)

In [8]:
customers.isnull().sum()

customerid      0
DOB             0
gender          0
address         1
Website       536
job             0
industry        0
dtype: int64

Với 1 bản ghi bị thiếu giá trị bị thiếu thông tin address có thể do hệ thống hoặc nhân viên điền thiếu thống tin khách hàng, thường sẽ loại bỏ nếu số lượng missing ít, nhưng do ảnh hưởng của customer lên các bảng dữ liệu khác nên chúng em sẽ không remove bản ghi này đi, mà thay thế bằng giá trị có tần suất nhiều nhất trong cột address

In [9]:
customers.loc[customers['address'].isnull()]

,customerid,DOB,gender,address,Website,job,industry
2562,0001135922,1970-01-01 09:42:28,Nam,NaN,KH0104|50184,white collar,education


In [10]:
mode_value = customers['address'].mode()[0]
# customers['address'].fillna('data missing', inplace=True)
customers['address'].fillna(mode_value, inplace=True)
customers.isnull().sum()

customerid      0
DOB             0
gender          0
address         0
Website       536
job             0
industry        0
dtype: int64

In [11]:
customers['Website'].head()

0     KH0104|0345
1     KH0104|0333
2     KH0104|0255
3     KH0104|0293
4    KH0104|40580
Name: Website, dtype: object

In [12]:
customers.loc[customers['Website'].isnull()]

,customerid,DOB,gender,address,Website,job,industry
7,0000000228,1970-01-01 09:43:32,Nam,"Ngũ Hành Sơn, ĐN",NaN,specialist,health service
14,0000000328,1970-01-01 09:33:41,Nam,Le Tan Trung,NaN,blue collar,economics
33,0000000777,1970-01-01 09:37:55,Nữ,THANH KHÊ,NaN,specialist,economics
49,0000003225,1970-01-01 09:20:04,Nam,ngũ hành sơn,NaN,blue collar,social service
57,0000003363,1970-01-01 09:52:10,Nam,ĐỐNG ĐA,NaN,student,government agent
...,...,...,...,...,...,...,...
4471,KH9830912,1970-01-01 10:09:52,Nữ,"ĐIỆN BÀN, QUẢNG NAM",NaN,teenager,no industry
4472,KH9834570,1970-01-01 09:50:41,Nam,"Ngu Hanh Son, Da Nang",NaN,student,social service
4473,KH984255,1970-01-01 10:10:50,Nam,"Hai Chau, Da Nang",NaN,teenager,no industry
4474,KH9855766,1970-01-01 09:41:25,Nữ,"A, THUAN PHUOC,HAI CHAU, DA NANG",NaN,specialist,finance


In [13]:
print(f'Bf drop duplicate records in Customer sheet: {customers.shape}')
customers.drop_duplicates().shape

Bf drop duplicate records in Customer sheet: (4479, 7)


(4479, 7)

### Ticket table

Với cột dữ liệu cashier có giá trị là 'Website' đều bị null ở các cột orderid, saledate, orderid, total, popcorn

In [16]:
ticket.isnull().sum()

orderid         96
cashier          0
saledate        96
total           96
customerid       0
ticketcode       0
date             0
time             0
slot             0
room             0
film             0
slot type        0
ticket type      0
ticket price     0
popcorn         96
dtype: int64

In [14]:
ticket.loc[ticket['cashier']=='Website']

,orderid,cashier,saledate,total,customerid,ticketcode,date,time,slot,room,film,slot type,ticket type,ticket price,popcorn
23,NaN,Website,NaT,NaN,0000000228,20100926,2019-05-25,19:45:00,G09,3,ALADDIN (PD) (G),ĐƠN,Thành viên,45000,NaN
24,NaN,Website,NaT,NaN,0000000228,20100927,2019-05-25,19:45:00,G10,3,ALADDIN (PD) (G),ĐƠN,Thành viên,45000,NaN
580,NaN,Website,NaT,NaN,0000011318,20092413,2019-05-18,18:40:00,G09,1,POKÉMON: THÁM TỬ PIKACHU (C13),ĐƠN,Thành viên,45000,NaN
581,NaN,Website,NaT,NaN,0000011318,20092412,2019-05-18,18:40:00,G08,1,POKÉMON: THÁM TỬ PIKACHU (C13),ĐƠN,Thành viên,45000,NaN
582,NaN,Website,NaT,NaN,0000011318,20092411,2019-05-18,18:40:00,G07,1,POKÉMON: THÁM TỬ PIKACHU (C13),ĐƠN,Thành viên,45000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23102,NaN,Website,NaT,NaN,0003008617,20106999,2019-05-30,21:35:00,H09,4,CHÚA TỂ GODZILLA: ĐẾ VƯƠNG BẤT TỬ (C13),ĐƠN,Thành viên,45000,NaN
23103,NaN,Website,NaT,NaN,0003008617,20106998,2019-05-30,21:35:00,H08,4,CHÚA TỂ GODZILLA: ĐẾ VƯƠNG BẤT TỬ (C13),ĐƠN,Thành viên,45000,NaN
23106,NaN,Website,NaT,NaN,0003008617,20091106,2019-05-17,20:15:00,F14,2,JOHN WICK 3: CHUẨN BỊ CHIẾN TRANH (C18),ĐƠN,Thành viên,45000,NaN
23107,NaN,Website,NaT,NaN,0003008617,20091105,2019-05-17,20:15:00,F13,2,JOHN WICK 3: CHUẨN BỊ CHIẾN TRANH (C18),ĐƠN,Thành viên,45000,NaN


- Có thể là do website này ko có thông tin về các trường orderid và saledate, và chỉ tính đơn hàng theo từng vé nên tổng total đều null. Tiếp theo là vì mua online nên là phần popcorn không được tính kèm vào đơn hàng.
    + Với cách xử lý là với trường total để bằng trường ticket price.
    + Với cột popcorn fill null là Không
    + Còn orderid với saledate không có vẫn dữ nguyên như cũ

In [15]:
ticket['total'].fillna(ticket['ticket price'], inplace=True)
ticket['popcorn'].fillna('Không', inplace=True)

In [19]:
ticket.isnull().sum()

orderid         96
cashier          0
saledate        96
total            0
customerid       0
ticketcode       0
date             0
time             0
slot             0
room             0
film             0
slot type        0
ticket type      0
ticket price     0
popcorn          0
dtype: int64

In [16]:
print(f'Bf drop duplicate records in Ticket sheet: {ticket.shape}')
ticket.drop_duplicates().shape

Bf drop duplicate records in Ticket sheet: (35474, 15)


(35474, 15)

### Film table

Sử dụng thư viện lấy data từ trang web IMDB về các thông tin phim còn thiếu, và tiến hành thêm thông tin phim còn thiếu

In [17]:
film.isnull().sum()

show_id          0
title            0
director        10
cast             6
country          7
release_year     0
rating           1
duration         0
listed_in        0
description      0
dtype: int64

In [18]:
film.head()

,show_id,title,director,cast,country,release_year,rating,duration,listed_in,description
0,s27,X-Men: Dark Phoenix,Simon Kinberg,"James McAvoy, Michael Fassbender, Jennifer Law...","United States, Canada",2019,PG-13,115 min,"Action-Adventure, Family, Science Fiction",When Jean Grey transforms into the Dark Phoeni...
1,s165,The Kid Who Would Be King,Joe Cornish,"Louis Ashbourne Serkis, Tom Taylor, Rebecca Fe...","United Kingdom, United States",2019,PG,121 min,"Action-Adventure, Coming of Age, Family","When a kid discovers the legendary sword, Exca..."
2,s194,Disney My Music Story: Perfume,Kentaro Takayanagi,"a-chan , KASHIYUKA , NOCCHi",NaN,2019,NaN,53 min,"Biographical, Documentary, Music",J-Pop band Perfume shares their passion for mu...
3,s222,Disney My Music Story: Yoshiki,Kentaro Takayanagi,YOSHIKI,"United States, Japan",2019,TV-PG,48 min,"Biographical, Documentary, Music",Yoshiki from â€œX Japanâ€ performs two Disney...
4,s241,Great Shark Chow Down,Mark Woodward,Dan Nachtrab,NaN,2019,TV-PG,45 min,"Animals & Nature, Documentary",Great Shark Chow Down â€“ prepare for a feast ...


In [19]:
film.loc[film['director'].isnull()]

,show_id,title,director,cast,country,release_year,rating,duration,listed_in,description
5,s265,Man vs. Shark,NaN,"Jeremiah Sullivan, Dave Hoffman",United States,2019,TV-PG,45 min,"Animals & Nature, Documentary",Marine biologist attempts to get bitten by a d...
9,s313,Oil Spill of The Century,NaN,LoÃ¯ck Peyron,NaN,2019,TV-PG,45 min,"Documentary, Historical",LoÃ¯ck Peyron investigates the 1978 Amoco Cadi...
12,s366,Star Wars: Galaxy's Edge-Adventure Awaits,NaN,"Neil Harris, Kaley Cuoco, Oscar Isaac, Sarah H...",United States,2019,TV-PG,81 min,"Documentary, Family",Celebrating Disneyâ€™s new land that brings St...
13,s393,Lost City of Machu Picchu,NaN,Craig Sechler,United States,2019,TV-14,44 min,"Documentary, Family",Who were the mysterious people who built Machu...
15,s440,Women Of Impact: Changing The World,NaN,Julianna Margulies,United States,2019,TV-PG,44 min,"Documentary, Historical",How will you make the world a better place? He...
27,s594,Marvel Rising: Battle of The Bands,NaN,"Dee Baker, Dove Cameron, Skai Jackson, Kathree...",United States,2019,TV-Y7,23 min,"Action-Adventure, Animation, Superhero",Gwen (Ghost-Spider) has to defeat Screaming Mi...
37,s620,Marvel Rising: Chasing Ghosts,NaN,"Dee Baker, Chloe Bennet, Dove Cameron, Skai Ja...",United States,2019,TV-Y7-FV,24 min,"Action-Adventure, Animation, Superhero",Ghost-Spider teams up with the Secret Warriors.
38,s621,Marvel Rising: Heart of Iron,NaN,"Dee Baker, Chloe Bennet, Dove Cameron, Robbie ...",United States,2019,TV-Y7,46 min,"Action-Adventure, Animation, Coming of Age",Ironheart joins the Secret Warriors to defend ...
42,s632,"Happy Birthday, Mickey!",NaN,NaN,NaN,2019,TV-G,2 min,"Animation, Comedy, Dance","Celebrating Mickey, the worldâ€™s most beloved..."
53,s963,Marvel Studios: Expanding the Universe,NaN,NaN,NaN,2019,TV-PG,13 min,"Documentary, Family",An exciting look into the future of Marvel Stu...


In [173]:
film['list_country'] = film['country'].str.split(',')
film['list_cast'] = film['cast'].str.split(',')
film['listed_in'] = film['listed_in'].str.split(',')
film['list_director'] = film['director'].str.split(',')

In [25]:
# def fill_film_data_missing(df):
#     ia = Cinemagoer()
#     for col in df.isnull().sum().items():
#         if col[1] > 0: 
#             for index in df[df[col[0]].isnull()].index.tolist():
#                 movies = ia.search_movie(df.iloc[index]['title'])
#                 max=0;idx_movies=0
#                 for i in range(len(movies)):
#                     temp = fuzz.token_sort_ratio(df.iloc[index]['title'],movies[i]['title'])
#                     if temp > max:
#                         max = temp
#                         idx_movies = i
#                 movie = ia.get_movie(movies[idx_movies].movieID)
#                 if (df.isnull().iloc[index]['director'])&('director' in movie.data.keys()):
#                     df.at[index,'director'] = movie.data['director'][0]['name']
#                 if (df.isnull().iloc[index]['list_country'])&('countries' in movie.data.keys()):
#                     df.at[index, 'list_country'] = movie.data['countries']
#                 if (df.isnull().iloc[index]['list_cast'])&('cast' in movie.data.keys()):
#                     df.at[index, 'list_cast'] = [movie.data['cast'][i]['name'] for i in range(len(movie['cast']))]
#                 if (df.isnull().iloc[index]['rating'])&('certificates' in movie.data.keys()):
#                     if len(movie.data['certificates'])>2:
#                         df.at[index, 'rating'] = re.sub(
#                             r'^.*?:', '', movie.data['certificates'][1])
#                     elif len(movie.data['certificates'])==2:
#                         df.at[index, 'rating'] = re.sub(
#                             r'^.*?:', '', movie.data['certificates'][0])
#     return df
# film = fill_film_data_missing(film)

In [26]:
# translator = Translator()
# for index1, row1 in ticket.iterrows():
#     for index2, row2 in film.iterrows():
#         translate_text = translator.translate(row2['title'].lower(), dest='en')
#         if SequenceMatcher(a=row1['film'].lower(), b=translate_text.text).ratio() >= 0.9:
#             ticket.at[index1, 'id_film'] = row2['show_id']

In [134]:
def fill_film_data_missing(film):
    #filling data film mising
    tmdb = TMDb(key="710d86a93f14fbb9e075cf6cddafb5f7")
    for col in film.isnull().sum().items():
        if col[1] > 0: 
            for index in film[film[col[0]].isnull()].index.tolist():
                results = tmdb.search().multi(film.iloc[index]['title'])
                max=90; movie=np.nan
                for result in results:
                    if result.is_movie():
                        if len(results)>1:
                            temp_movie = tmdb.movie(result.id)
                            temp = fuzz.token_sort_ratio(
                                film.iloc[index]['title'].lower(), temp_movie.details().title.lower())
                            if temp > max:
                                max = temp
                                movie = temp_movie
                        else:
                            movie = tmdb.movie(result.id)
            if (movie != np.nan) & (type(movie) != float):
                if film.isnull().iloc[index]['original_title']:
                    film.at[index, 'original_title'] = movie.details().original_title
                if film.isnull().iloc[index]['list_director']:
                    film.at[index, 'list_director'] = [person for person in movie.credits().crew if person.job == 'Director']
                if film.isnull().iloc[index]['list_country']:
                    film.at[index, 'list_country'] = [country for country in movie.details().production_countries]
                if film.isnull().iloc[index]['list_cast']:
                    if len(movie.credits().cast) > 5:
                        film.at[index, 'list_cast'] = [cast for cast in movie.credits().cast[:5]]
                    elif len(movie.credits().cast) <= 5:
                        film.at[index, 'list_cast'] = [cast for cast in movie.credits().cast]
                if film.isnull().iloc[index]['rating']:
                    for release_date in movie.release_dates().results:
                        if release_date.iso_3166_1 == 'VI':
                            for rl_local in release_date.release_dates:
                                if rl_local.certification != '':
                                    film.at[index,'rating'] = rl_local.certification
                                    break
                        elif release_date.iso_3166_1 == 'US':
                            for rl_local in release_date.release_dates:
                                if rl_local.certification != '':
                                    film.at[index,'rating'] = rl_local.certification
                                    break
                if film.isnull().iloc[index]['production_companies']:
                    for info in movie.details().production_companies:
                        production_companies_name = info.name
                        film.at[index, 'production_companies'] = production_companies_name
                        
                if film.isnull().iloc[index]['release_date']:
                    film.at[index, 'release_date'] = movie.details().release_date
                
                if film.isnull().iloc[index]['poster_path']:
                    if (movie.details().poster_path != np.nan) | (movie.details().poster_path is not None):
                        film.at[index, 'poster_path'] = 'https://image.tmdb.org/t/p/w500' + \
                        movie.details().poster_path
                if film.isnull().iloc[index]['revenue']:
                    film.at[index, 'revenue'] = movie.details().revenue
                if film.isnull().iloc[index]['vote_average']:
                    film.at[index, 'vote_average'] = movie.details().vote_average
                if film.isnull().iloc[index]['vote_count']:
                    film.at[index, 'vote_average'] = movie.details().vote_count
    return film

In [137]:
def get_film_data_missing(search_text):
#getting data film mising
    tmdb = TMDb(key="710d86a93f14fbb9e075cf6cddafb5f7", language='vi')
    results = tmdb.search().multi(search_text)
    max=50; new_row={}; movie=np.nan
    for result in results:
        if result.is_movie():
            if len(results)>1:
                temp_movie = tmdb.movie(result.id)
                temp = fuzz.token_sort_ratio(
                    search_text.lower(), temp_movie.details().title.lower())
                if temp > max:
                    max = temp
                    movie = temp_movie
            else:
                movie = tmdb.movie(result.id)
    if (movie != np.nan) & (type(movie) != float):
        new_row = {'show_id': movie.details().id}
        new_row.update({'original_title': movie.details().original_title,
                        'title': movie.details().title,
                        'list_director': [person.name  for person in movie.credits().crew if person.job == 'Director']})
        if len(movie.credits().cast) > 5:
            new_row.update(
                {'list_cast': [cast.name for cast in movie.credits().cast[:5]]})
        else:
            new_row.update(
                {'list_cast': [cast.name for cast in movie.credits().cast]})
        for release_date in movie.release_dates().results:
            if release_date.iso_3166_1 == 'VI':
                for rl_local in release_date.release_dates:
                    if rl_local.certification != '':
                        new_row.update({'rating': rl_local.certification})
                        break
            elif (release_date.iso_3166_1 == 'US')&('rating' not in new_row.keys()):
                for rl_local in release_date.release_dates:
                    if rl_local.certification != '':
                        new_row.update({'rating': rl_local.certification})
                        break
        for info in movie.details().production_companies:
            production_companies_name = info.name
            new_row.update({'production_companies': production_companies_name})
        new_row.update({'list_country': [country.name for country in movie.details().production_countries], 
                        'release_year': movie.details().year,
                        'release_date': movie.details().release_date,
                        'duration': str(movie.details().runtime) + ' min',
                        'listed_in': [genres.name for genres in movie.details().genres],
                        'description':movie.details().overview,
                        'poster_path': 'https://image.tmdb.org/t/p/w500'+movie.details().poster_path,
                        'revenue': movie.details().revenue,
                        'vote_average': movie.details().vote_average,
                        'vote_count': movie.details().vote_count})
        # ticket.at[idx, 'show_id'] = new_row['show_id']
    return new_row

In [29]:
# async def fill_and_get_film_data_missing(film, ticket):
#     # filling data film mising
#     tmdb = aioTMDb(key="710d86a93f14fbb9e075cf6cddafb5f7")
#     for col in film.isnull().sum().items():
#         if col[1] > 0:
#             for index in film[film[col[0]].isnull()].index.tolist():
#                 results = await tmdb.search().multi(film.iloc[index]['title'])
#                 max = 95
#                 for result in results:
#                     if result.is_movie():
#                         temp_movie = await tmdb.movie(result.id)
#                         temp_movie_details = await temp_movie.details()
#                         temp = fuzz.token_sort_ratio(
#                             film.iloc[index]['title'].lower(), temp_movie_details.original_title.lower())
#                         if temp > max:
#                             max = temp
#                             movie = temp_movie
#                             movie_credits = await movie.credits()
#                             movie_details = await movie.details()
#                 if film.isnull().iloc[index]['list_director']:
#                     film.at[index, 'list_director'] = [
#                         person for person in movie_credits.crew if person.job == 'Director']
#                 if film.isnull().iloc[index]['list_country']:
#                     film.at[index, 'list_country'] = [
#                         country for country in movie_details.production_countries]
#                 if film.isnull().iloc[index]['list_cast']:
#                     if len(movie_credits.cast) > 5:
#                         film.at[index, 'list_cast'] = [
#                             cast for cast in movie_credits.cast[:5]]
#                     elif len(movie_credits.cast) <= 5:
#                         film.at[index, 'list_cast'] = [
#                             cast for cast in movie_credits.cast]
#                 film[index, 'release_date'] = movie_details.release_date
#     # getting data film mising
#     translator = Translator()
#     for idx, title in ticket.film.items():
#         translate_text = translator.translate(title, dest='en')
#         results = await tmdb.search().multi(translate_text.text.lower())
#         max = 95
#         new_row = {}
#         for result in results:
#             if result.is_movie():
#                 temp_movie = await tmdb.movie(result.id)
#                 temp_movie_details = await temp_movie.details()
#                 temp = fuzz.token_sort_ratio(
#                     translate_text.text.lower(), temp_movie_details.original_title.lower())
#                 if temp > max:
#                     max = temp
#                     movie = temp_movie
#                     movie_credits = await movie.credits()
#                     movie_details = await movie.details()
#         if movie is not None:
#             new_row = {'show_id': 's' +
#                        str(int(film['show_id'].iloc[-1].split('s')[1])+1)}
#             if movie_details.original_title not in film.title:
#                 new_row.update({'title': movie_details.original_title,
#                                 'list_director': [person for person in movie_credits.crew if person.job == 'Director']})
#                 if len(movie_credits.cast) > 5:
#                     new_row.update(
#                         {'list_cast': [cast for cast in movie_credits.cast[:5]]})
#                 else:
#                     new_row.update(
#                         {'list_cast': [cast for cast in movie_credits.cast]})
#                 new_row.update({'list_country': [country for country in movie_details.production_countries],
#                                 'release_year': movie_details.year,
#                                 'release_date': movie_details.release_date,
#                                 'duration': movie_details.runtime + 'min',
#                                 'listed_in': [genres for genres in movie_details.genres],
#                                 'description': movie_details.overview})
#             ticket.at[idx, 'show_id'] = new_row['show_id']
#             df_the_dict = pd.DataFrame(
#                 {'name': new_row.keys(), 'value': new_row.values()})
#             film = pd.concat([film, df_the_dict], ignore_index=True)
#     return film

In [149]:
def fill_missing_by_cinemagoer(film):
    # filling data film mising
    try:
        ia = Cinemagoer()
        for col in film.isnull().sum().items():
            if col[1] > 0:
                for index in film[film[col[0]].isnull()].index.tolist():
                    movies = ia.search_movie(film.iloc[index]['title'].lower())
                    max = 95
                    idx_movies = 0
                    for i in range(len(movies)):
                        temp = fuzz.token_sort_ratio(
                            film.iloc[index]['title'].lower(), movies[i]['title'].lower())
                        if temp > max:
                            max = temp
                            idx_movies = i
                    movie = ia.get_movie(movies[idx_movies].movieID)
                    if (film.isnull().iloc[index]['rating']) & ('certificates' in movie.data.keys()):
                        certificates=[]
                        for cer in movie.data['certificates']:
                            certificates.append(cer.split(':'))
                        for cer in certificates:
                            if cer[0]=='Vietnam':
                                film.at[index, 'rating'] = cer[1]
                    if (film.isnull().iloc[index]['list_country']) & ('countries' in movie.data.keys()):
                        film.at[index, 'list_country'] = movie.data['countries']
                    if (film.isnull().iloc[index]['list_cast']) & ('cast' in movie.data.keys()):
                        film.at[index, 'list_cast'] = [cast['name']
                                                          for cast in movie.data['cast']]
                    if (film.isnull().iloc[index]['list_director']) & ('director' in movie.data.keys()):
                        film.at[index, 'list_director'] = [director['name']
                                                       for director in movie.data['director']]
                    if (film.isnull().iloc[index]['vote_count']) & ('votes' in movie.data.keys()):
                        film.at[index, 'vote_count'] = movie.data['votes']
                    if (film.isnull().iloc[index]['poster_path']) & ('cover url' in movie.data.keys()):
                        film.at[index, 'poster_path'] = movie.data['cover url']
                    if (film.isnull().iloc[index]['production_companies']) & ('cast' in movie.data.keys()):
                        film.at[index, 'production_companies'] = movie.data['production companies'][0]['name']
    except IMDbError as e:
        print(e)
    return film

In [66]:
ticket.film.unique()

array(['AVENGERS: HỒI KẾT', 'JOHN WICK 3: CHUẨN BỊ CHIẾN TRANH (C18)',
       'NGÔI ĐỀN KỲ QUÁI (C18)', 'VÔ GIAN ĐẠO (C18)',
       'MẸ MA THAN KHÓC LA LLORONA (C18)', 'ALADDIN (PD) (G)',
       'POKÉMON: THÁM TỬ PIKACHU (C13)', 'ALADDIN (LT) (G)',
       'THẰNG EM LÝ TƯỞNG', 'ƯỚC HẸN MÙA THU', 'QUÝ CÔ LỪA ĐẢO',
       'CHÚA TỂ GODZILLA: ĐẾ VƯƠNG BẤT TỬ (C13)',
       '(LT) DORAEMON: NOBITA VÀ MẶT TRĂNG PHIÊU LƯU KÝ (G)',
       'LẬT MẶT: NHÀ CÓ KHÁCH', 'VỢ BA', 'NỤ HÔN MA QUÁI (C18)',
       '(PĐ) DORAEMON: NOBITA VÀ MẶT TRĂNG PHIÊU LƯU KÝ (G)',
       'CORGI: NHỮNG CHÚ CHÓ HOÀNG GIA (LT)', 'CÀ CHỚN ANH ĐỪNG ĐI'],
      dtype=object)

In [114]:
dict1=get_film_data_missing('john wick 3')

In [178]:
for col in dict1.keys():
    if col not in film.columns:
        film[col] = np.nan

In [34]:
# pattern = re.compile(r'\(([^)]+)\)')
# ticket['rating'] = ticket['film'].apply(lambda x: pattern.findall(x)[0] if pattern.findall(x) != [] else (pattern.findall(x)[1] if len(pattern.findall(x))==2 else np.nan))

In [116]:
ticket['title_clean'] = ticket['film'].apply(
    lambda x: re.sub(r'\(([^)]+)\)', '', x).strip())

In [179]:
for title in ticket.title_clean.unique():
    dict_temp = get_film_data_missing(title)
    if (dict_temp != np.nan) | (dict_temp!={}):
        if dict_temp['title'] not in film.title:
            film.loc[len(film)] = dict_temp
            ticket.loc[ticket.title_clean == title,'show_id'] = dict_temp['show_id']
    else:
        print(title)

In [180]:
ticket.loc[ticket['show_id'].isna() == True].title_clean.unique()

array([], dtype=object)

In [71]:
tmdb = TMDb(key="710d86a93f14fbb9e075cf6cddafb5f7", language='vi')
results = tmdb.search().multi('ngôi đền kỳ quái')
for result in results:
    if result.is_movie():
        movie = tmdb.movie(result.id)
        print(movie.details().original_title,
              movie.details().belongs_to_collection.poster_path)
        print(movie.details().overview, "\n")

พี่นาค 2 /6RI0xVRqmUGwSouUm70KZVOkh69.jpg
Bộ đôi khẩu nghiệp đã hoàn tục quay lại con đường tạo nghiệp nhưng không may quả báo đến sớm, cặp đôi trời đánh suýt chết nhiều lần bởi lời nguyền bí ẩn. Họ đã phải quay lại đền tìm Nhong để trợ giúp phá giải lời nguyền. Liệu họ sẽ tìm được cách phá giải lời nguyền hay khiến cho mọi thứ trở nên tồi tệ hơn. 

พี่นาค /6RI0xVRqmUGwSouUm70KZVOkh69.jpg
Ngôi Đền Kỳ Quái nói về bộ ba hoàn cảnh vì lời thề ngớ ngẩn sẽ cạo đầu đi tu nếu trúng số và may mắn trúng ngay giải chót. Và “may mắn” lại nhân đôi khi hội bạn thân lại chọn trúng ngay ngôi đền bị ám bởi hàng loạt hiện tượng siêu nhiên. Từ một lời thề quái gở, hội bạn “thân ai nấy lo” này sẽ phải đối mặt với hàng loạt rắc rối đang đón chờ phía trước ra sao? 

พี่นาค 3 /6RI0xVRqmUGwSouUm70KZVOkh69.jpg
Peace Aod, một người thợ bốc mộ trẻ tuổi ở Đền Thamma Nakanimitr, đã khai quật được một chiếc vòng chân cổ bằng vàng từ một ngôi mộ từ thời Thế chiến thứ hai. Anh ta quyết định bỏ túi chiếc vòng chân và 

In [182]:
film=fill_film_data_missing(film)

In [ ]:
film=fill_missing_by_cinemagoer(film)

In [190]:
film.isnull().sum()

show_id                  0
title                    0
director                27
cast                    23
country                 24
release_year             0
rating                   3
duration                 0
listed_in                0
description              0
list_country             3
list_cast                0
list_director            2
original_title          51
production_companies    43
release_date            51
poster_path             42
revenue                 52
vote_average            52
vote_count              49
dtype: int64

In [192]:
film_copy = film.copy()

In [203]:
film.rating.unique()

array(['PG-13', 'PG', nan, 'TV-PG', 'TV-G', 'TV-14', 'G', 'TV-Y7',
       'TV-Y7-FV', 'R', 'C18', 'P', 'C16'], dtype=object)

In [101]:
# def flat_data(data, column, new_name, drop_columns):
#     flat_film_data = [(index, item) for index, sublist in data.loc[data[column].notnull(
#     )][column].items() for item in sublist]
#     flat_film_data = pd.DataFrame(flat_film_data,
#                                   columns=['index', new_name]).set_index('index')
#     data = data.drop(drop_columns,
#                                    axis=1).join(flat_film_data)
#     return data
    

In [197]:
# film['list_country'] = film['list_country'].loc[film['list_country'].notnull()].apply(lambda cell:
#                                                   ''.join(c for c in cell if c not in "'[]").split(', '))
# film['list_cast'] = film['list_cast'].loc[film['list_cast'].notnull()].apply(lambda cell:
#                                                                                       ''.join(c for c in cell if c not in "'[]").split(', '))
film['listed_in'] = film['listed_in'].loc[film['listed_in'].notnull()].apply(lambda cell:
                                                                                      ''.join(c for c in cell if c not in "'[]").split(', '))
# film['list_director'] = film['list_director'].loc[film['list_director'].notnull()].apply(lambda cell:
#                                                                                       ''.join(c for c in cell if c not in "'[]").split(', '))

In [102]:
# film = flat_data(film, 'list_country', 'country', ['country', 'list_country'])
# film = flat_data(film, 'list_cast', 'cast', ['cast', 'list_cast'])
# film = flat_data(film, 'list_director', 'director',
#                  ['director', 'list_director'])

In [207]:
flat_country_film_data = [(index, item) for index, sublist in film.loc[film['list_country'].notnull(
)]['list_country'].items() for item in sublist]
flat_cast_film_data = [(index, item) for index, sublist in film.loc[film['list_cast'].notnull(
)]['list_cast'].items() for item in sublist]
flat_genres_film_data = [(index, item) for index, sublist in film.loc[film['listed_in'].notnull(
)]['listed_in'].items() for item in sublist]
flat_director_film_data = [(index, item) for index, sublist in film.loc[film['list_director'].notnull(
)]['list_director'].items() for item in sublist]

In [208]:
film = film.drop(['country', 'list_country'],
                 axis=1).join(pd.DataFrame(flat_country_film_data,
                                           columns=['index', 'country']).set_index('index'))
film = film.drop(['cast', 'list_cast'],
                 axis=1).join(pd.DataFrame(flat_cast_film_data,
                                           columns=['index', 'cast']).set_index('index'))
film = film.drop('listed_in', axis=1).join(pd.DataFrame(flat_genres_film_data,
                                                        columns=['index', 'genres']).set_index('index'))
film = film.drop(['director', 'list_director'],
                 axis=1).join(pd.DataFrame(flat_director_film_data,
                                           columns=['index', 'director']).set_index('index'))

In [209]:
film.isnull().sum()

show_id                   0
title                     0
release_year              0
rating                   64
duration                  0
description               0
original_title          932
production_companies    773
release_date            932
poster_path             771
revenue                 935
vote_average            935
vote_count              818
country                  31
cast                      6
genres                    0
director                 19
dtype: int64

In [212]:
film.dtypes

show_id                  object
title                    object
release_year              int64
rating                   object
duration                 object
description              object
original_title           object
production_companies     object
release_date             object
poster_path              object
revenue                 float64
vote_average            float64
vote_count              float64
country                  object
cast                     object
genres                   object
director                 object
dtype: object

In [218]:
ticket_film = pd.merge(ticket, film, on='show_id')

In [219]:
ticket_film.isnull().sum()

orderid                 1975
cashier                    0
saledate                1975
total                      0
customerid                 0
ticketcode                 0
date                       0
time                       0
slot                       0
room                       0
film                       0
slot type                  0
ticket type                0
ticket price               0
popcorn                    0
title_clean                0
show_id                    0
title                      0
release_year               0
rating                  6595
duration                   0
description                0
original_title             0
production_companies       0
release_date               0
poster_path                0
revenue                    0
vote_average               0
vote_count                 0
country                 6435
cast                       0
genres                     0
director                   0
dtype: int64

In [222]:
with pd.ExcelWriter('DATA SET - VÒNG 1 CUỘC THI DATA GOT TALENT 2023 CLEAN VER5.xlsx') as writer:
    customers.to_excel(writer, sheet_name='customer', index=False)
    ticket.to_excel(writer, sheet_name='ticket', index=False)
    film.to_excel(writer, sheet_name='film', index=False)

In [223]:
ticket_film.to_excel('MAP_TICKET_AND_FILM.xlsx', index=False)